In [9]:
# 读入表格 res_Albumin.csv

import pandas as pd
import numpy as np
import os
import sys
import math

result_prefix = "./result/statistics_r/res/time"
df_origin = pd.read_csv(f'{result_prefix}/res_Albumin.csv',index_col=0)


# Format the result of stage2

In [10]:
df_origin

,diet,stage,round1,round2,round3,p,p_nonlinear,d1,d2,d3
1,water,0,10.76,13.72,-1.0,0.1188,0.1491,down,up,-1
2,protein,0,52.23,52.46,-1.0,0.0036,0.0012,down,up,-1
3,fat,0,5.27,5.85,-1.0,0.0000,0.0003,down,up,-1
4,carbohydrate,0,194.82,286.79,-1.0,0.0019,0.0007,down,up,-1
5,df,0,9.44,14.23,-1.0,0.0002,0.0001,down,up,-1
...,...,...,...,...,...,...,...,...,...,...
96,riboflavin,3,0.72,0.77,-1.0,0.0173,0.0208,down,up,-1
97,nicotinic,3,11.57,17.42,-1.0,0.0000,0.0000,down,up,-1
98,ascorbic,3,79.27,-1.00,-1.0,0.2656,0.2942,down,-1,-1
99,dpi,3,8.75,10.54,-1.0,0.0010,0.0003,down,up,-1


In [11]:


#---------------------------------
# 增加一列，边界boundary，字符串形式
units_list ={'water':'g/d', 'protein':'g/d', 'fat':'g/d', 'carbohydrate':'g/d', 'Calories':'kcal/d', 'df':'g/d', 
'k':'mg/d', 'na':'mg/d', 'mg':'mg/d', 'ca':'mg/d', 'p':'mg/d', 'fe':'mg/d', 'zn':'mg/d', 'cu':'mg/d', 'mn':'mg/d', 'se':'mg/d', 'retinol':'ug/d', 'vitaminA':'ugRAE/d', 'carotene':'ug/d', 
'vitaminE':'mg/d', 'thiamine':'mg/d', 'riboflavin':'mg/d', 'nicotinic':'mg/d', 'ascorbic':'mg/d', 'dpi':'g/kg/d','dei':'kcal/kg/d'}
names_list = {'water':'Water', 'protein':'Protain', 'fat':'Fat', 'carbohydrate':'Carbohydrate', 'Calories':'Calories', 'df':'Df', 'k':'K', 'na':'Na', 'mg':'Mg', 
              'ca':'Ca', 'p':'P', 'fe':'Fe', 'zn':'Zn', 'cu':'Cu', 'mn':'Mn', 
              'se':'Se', 'retinol':'Retinol', 'vitaminA':'Vitamin A', 'carotene':'Carotene', 'vitaminE':'Vitamin E', 'thiamine':'Thiamine', 'riboflavin':'Riboflavin', 
              'nicotinic':'Nicotinic', 'ascorbic':'Ascorbic', 'dpi':'DPI', 'dei':'DEI'}


div_dict = {'water':100,'protein':1,'fat':10,
                'carbohydrate':1,'Calories':10,'df':1,'k':10,'na':100,
                'mg':10,'ca':100,'p':10,'fe':1,'zn':1,'cu':1,
                'mn':1,'se':1,'retinol':10,'vitaminA':100,
                'carotene':100,'vitaminE':1,'thiamine':1,'riboflavin':1,
                'nicotinic':1,'ascorbic':1,'dpi':0.1,'dei':1,'hsCRP':1}

# 修改units_list
for key in units_list.keys():
    cur_unit = units_list[key]
    if(div_dict[key] != 1):
        units_list[key] =  f'{div_dict[key]}{cur_unit}'

# 遍历每一行，计算边界
for index, row in df_origin.iterrows():
    # print(index)
    # 如果round1 = -1，boundary = ‘-’
    # 如果round2 = -1，boundary = (round1, -)
    # 如果round3 = -1，boundary = (round1, round2)
    # 如果round3 != -1，boundary = (round1, round2) and (round3, -)
    # 保留两位小数
    if(row['round1'] == -1):
        boundary = '-'
    elif(row['round2'] == -1):
        boundary = '(' + f'{row["round1"]:.2f}' + ', -)'
    elif(row['round3'] == -1):
        boundary = '(' + f'{row["round1"]:.2f}' + ', ' + f'{row["round2"]:.2f}' + ')'
    else:
        boundary = '(' + f'{row["round1"]:.2f}' + ', ' + f'{row["round2"]:.2f}' + ') or (' + f'{row["round3"]:.2f}' + ', -)'
    df_origin.loc[index, 'boundary'] = boundary


# 增加一列p_str,对于p值，则 为*，如果p<0.001,则为***，如果p<0.01,则为**，如果p<0.05,否则为空,用pd方法实现
df_origin['p_str'] = df_origin['p'].apply(lambda x: '***' if x < 0.001 else ('**' if x < 0.01 else ('*' if x < 0.05 else '')))
# p_linear_str, 映射p_linear
df_origin['p_nonlinear_str'] = df_origin['p_nonlinear'].apply(lambda x: '***' if x < 0.001 else ('**' if x < 0.01 else ('*' if x < 0.05 else '')))
df_origin['unit'] = [units_list[i] for i in df_origin['diet']]

df_origin.to_csv(f'{result_prefix}/res_Albumin_new.csv')
df_origin.to_excel(f'{result_prefix}/res_Albumin_new.xlsx')

# --------------------------------------------
# copy表格，只保留列diet，stage，round1，round2，round3，p_str，p_linear_str
df_new = df_origin[['diet','unit','stage','round1','round2','round3','p_str','p_nonlinear_str']]
# 遍历每一行，round1，round2，round3，如果为-1，则置为 -
for index, row in df_new.iterrows():
    # 两位小数
    df_new.loc[index, 'round1'] = f'{row["round1"]:.2f}'
    df_new.loc[index, 'round2'] = f'{row["round2"]:.2f}'
    df_new.loc[index, 'round3'] = f'{row["round3"]:.2f}'
    if(int(row['round1']) == -1):
        df_new.loc[index, 'round1'] = '-'
    if(int(row['round2']) == -1):
        df_new.loc[index, 'round2'] = '-'
    if(int(row['round3']) == -1):
        df_new.loc[index, 'round3'] = '-'
df_new['diet'] = df_new['diet'].apply(lambda x : x.upper() if x == 'dpi' or x == 'dei' else x.capitalize())

df_new.to_excel(f'{result_prefix}/res_Albumin_new_str.xlsx')




df_subs = []
df_diet = df_new[['diet','unit']]
df_new = df_new.drop('diet',axis=1)
df_new = df_new.drop('unit',axis=1)
df_new = df_new.groupby('stage')

for index, df_sub in df_new:
    # print(index)
    # print(df_sub)
    df_sub = df_sub.drop('stage',axis=1)
    df_sub.columns = [str(index) + '_' + i for i in df_sub.columns]
    df_sub.index = range(len(df_sub))
    df_subs.append(df_sub)
df_diet = df_diet[:len(df_sub)]
df_diet.index = range(len(df_diet))
df_diet = pd.concat([df_diet] + df_subs, axis=1)


df_diet.to_excel(f'{result_prefix}/res_Albumin_finally.xlsx')


# ------------------------------------------
# 只保留列diet，stage，boundary，p_str，p_linear_str
df_new = df_origin[['diet','unit','stage','boundary','p_str','p_nonlinear_str']]
# 先去除列diet，保存到其他变量中
df_diet = df_new[['diet','unit']]
df_new = df_new.drop('diet',axis=1)
df_new = df_new.drop('unit',axis=1)
# 按照stage分组，作为新的列，如boundary0，p_str0，p_linear_str0，boundary1，p_str1，p_linear_str1，boundary2，p_str2，p_linear_str2
df_new = df_new.groupby('stage')
df_subs = []
for index, df_sub in df_new:
    # print(index)
    # print(df_sub)
    df_sub = df_sub.drop('stage',axis=1)
    df_sub.columns = [i+'_'+str(index) for i in df_sub.columns]
    df_sub.index = range(len(df_sub))
    df_subs.append(df_sub)
# df_diet的前len(df_sub)行
df_diet = df_diet[:len(df_sub)]
df_diet.index = range(len(df_diet))
# 横向合并df_new的各个分组
df_new = pd.concat([df_diet] + df_subs, axis=1)
print(df_new.shape)
# 保存表格
df_new.to_csv(f'{result_prefix}/res_Albumin_new_cut.csv')

(25, 14)


/home/v-wuyueying/miniconda3/envs/esrd/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/home/v-wuyueying/miniconda3/envs/esrd/lib/python3.7/site-packages/ipykernel_launcher.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [12]:
# 按照stage分组，作为新的列
df_origin_clone = df_origin.copy()
df_origin_clone['p_str'] = df_origin_clone['p'].apply(lambda x: '$<$0.001' if x < 0.001 else '$<$0.01' if x < 0.01 else f'{x:.2f}')
df_origin_clone['p_nonlinear_str'] = df_origin_clone['p_nonlinear'].apply(lambda x: '$<$0.001' if x < 0.001 else '$<$0.01' if x < 0.01 else f'{x:.2f}')
# 遍历每一行，如果round值为-1，则变为-
df_origin_clone['round1'] = df_origin_clone['round1'].apply(lambda x: '-' if x == -1 else f'{x:.2f}')
df_origin_clone['round2'] = df_origin_clone['round2'].apply(lambda x: '-' if x == -1 else f'{x:.2f}')
df_origin_clone['round3'] = df_origin_clone['round3'].apply(lambda x: '-' if x == -1 else f'{x:.2f}')
df_new = df_origin_clone.groupby('stage')
df_subs = []
for index, df_sub in df_new:
    df_sub = df_sub.drop(columns=['stage'],axis=1)
    df_sub.columns = [f'{x}_{index}' for x in df_sub.columns]
    df_sub.index = range(len(df_sub))
    df_subs.append(df_sub)
df_diet = df_origin_clone[:len(df_subs[0])][['diet','unit']]
# df_diet = df_diet.drop(columns=['stage'],axis=1)
df_diet.index = range(len(df_diet))
df_diet_new = pd.concat([df_diet] + df_subs,axis=1)
# print(df_diet_new)

# print to latex
for index, row in df_diet_new.iterrows():
    cur_diet_label = names_list[row['diet']]
    print(f'{cur_diet_label} & {row["unit"]} & {row["round1_3"]} & {row["round2_3"]} & {row["round3_3"]} & {row["p_str_3"]} & {row["p_nonlinear_str_3"]} & {row["round1_0"]} & {row["round2_0"]} & {row["round3_0"]} & {row["p_str_0"]} & {row["p_nonlinear_str_0"]} & {row["round1_1"]} & {row["round2_1"]} & {row["round3_1"]} & {row["p_str_1"]} & {row["p_nonlinear_str_1"]} & {row["round1_2"]} & {row["round2_2"]} & {row["round3_2"]} & {row["p_str_2"]} & {row["p_nonlinear_str_2"]} \\\\')

Water & 100g/d & 12.40 & 13.99 & - & 0.44 & 0.34 & 10.76 & 13.72 & - & 0.12 & 0.15 & 13.70 & - & - & 0.90 & 0.94 & 14.45 & 20.27 & - & 0.24 & 0.12 \\
Protain & g/d & 50.29 & 60.72 & - & $<$0.001 & $<$0.001 & 52.23 & 52.46 & - & $<$0.01 & $<$0.01 & 41.80 & 48.48 & - & 0.03 & 0.01 & 51.29 & 68.10 & - & 0.23 & 0.11 \\
Fat & 10g/d & - & - & - & $<$0.001 & $<$0.001 & 5.27 & 5.85 & - & $<$0.001 & $<$0.001 & 5.59 & 5.75 & - & $<$0.01 & $<$0.01 & 6.04 & 8.21 & - & 0.58 & 0.50 \\
Carbohydrate & g/d & 206.26 & 292.61 & - & 0.03 & 0.02 & 194.82 & 286.79 & - & $<$0.01 & $<$0.001 & 201.57 & - & - & 0.59 & 0.83 & 215.33 & - & - & 0.59 & 0.72 \\
Df & g/d & 9.73 & 14.50 & - & $<$0.001 & $<$0.001 & 9.44 & 14.23 & - & $<$0.001 & $<$0.001 & 9.20 & 9.33 & 14.89 & 0.17 & 0.20 & 10.17 & 14.63 & - & 0.01 & $<$0.01 \\
K & 10mg/d & 144.09 & 209.25 & - & $<$0.01 & $<$0.001 & 142.06 & 208.75 & - & 0.12 & 0.06 & 102.17 & 138.99 & - & 0.02 & 0.03 & 148.89 & 225.09 & - & 0.01 & 0.02 \\
Na & 100mg/d & 22.39 & 24.68 

# Format the result of stage1

In [13]:
# 读入表格 res_Albumin.csv

import pandas as pd
import numpy as np
import os
import sys
import math

df_cox_origin = pd.read_csv(f'{result_prefix}/res_cox_Albumin.csv',index_col=0)

In [14]:
df_cox_origin

,col,stage,c,c1,c2,z,p,951,952
1,water,0,0.045488,1.046538,0.040994,1.109624,0.267161,0.965742,1.134094
2,protein,0,-0.005254,0.994760,0.022682,-0.231647,0.816812,0.951505,1.039981
3,fat,0,0.329427,1.390171,0.157719,2.088688,0.036736,1.020510,1.893735
4,carbohydrate,0,-0.004403,0.995607,0.005114,-0.860960,0.389260,0.985677,1.005636
5,df,0,-0.045906,0.955132,0.057598,-0.797007,0.425447,0.853171,1.069278
...,...,...,...,...,...,...,...,...,...
96,riboflavin,3,0.013786,1.013881,0.012814,1.075818,0.282009,0.988734,1.039668
97,nicotinic,3,-0.038372,0.962355,0.029243,-1.312191,0.189456,0.908748,1.019123
98,ascorbic,3,-0.002349,0.997653,0.002015,-1.165759,0.243712,0.993720,1.001602
99,dpi,3,-0.047024,0.954065,0.079088,-0.594577,0.552127,0.817068,1.114032


In [15]:
# 按照stage分组，作为新的列，如c_0,c_1_0,c_2_0等
df_cox_origin['p_str'] = df_cox_origin['p'].apply(lambda x: '$<$0.001' if x < 0.001 else ('$<$0.01' if x < 0.01 else f'{x:.2f}'))
df_cox_new = df_cox_origin.groupby('stage')
df_subs = []
for index, df_sub in df_cox_new:
    # print(index)
    # print(df_sub)
    df_sub = df_sub.drop('stage',axis=1)
    df_sub.columns = [i+'_'+str(index) for i in df_sub.columns]
    df_sub.index = range(len(df_sub))
    df_subs.append(df_sub)
# df_diet的前len(df_sub)行
df_diet = df_diet[:len(df_sub)]
df_diet.index = range(len(df_diet))
# 横向合并df_new的各个分组
df_cox_new = pd.concat([df_diet] + df_subs, axis=1)
print(df_cox_new.shape)
# print(df_cox_new)

# out put table for latex , col, c1, f'{951}-{952}'(保留两位小数)，p
for index, row in df_cox_new.iterrows():
    cur_diet = row['diet']
    print(f"{names_list[cur_diet]} & {units_list[cur_diet]} & {row['c1_3']:.2f} & {row['951_3']:.2f}-{row['952_3']:.2f} & {row['p_str_3']} & {row['c1_0']:.2f} & {row['951_0']:.2f}-{row['952_0']:.2f} & {row['p_str_0']} & {row['c1_1']:.2f} & {row['951_1']:.2f}-{row['952_1']:.2f} & {row['p_str_1']} & {row['c1_2']:.2f} & {row['951_2']:.2f}-{row['952_2']:.2f} & {row['p_str_2']} \\\\")


(25, 38)
Water & 100g/d & 1.02 & 0.97-1.06 & 0.52 & 1.05 & 0.97-1.13 & 0.27 & 1.02 & 0.94-1.10 & 0.71 & 0.95 & 0.84-1.09 & 0.48 \\
Protain & g/d & 0.98 & 0.96-1.01 & 0.28 & 0.99 & 0.95-1.04 & 0.82 & 1.01 & 0.95-1.06 & 0.82 & 0.98 & 0.92-1.05 & 0.62 \\
Fat & 10g/d & 1.07 & 0.87-1.30 & 0.53 & 1.39 & 1.02-1.89 & 0.04 & 0.86 & 0.60-1.22 & 0.39 & 0.73 & 0.44-1.21 & 0.22 \\
Carbohydrate & g/d & 1.00 & 0.99-1.00 & 0.48 & 1.00 & 0.99-1.01 & 0.39 & 0.99 & 0.98-1.00 & 0.18 & 1.02 & 1.00-1.03 & 0.11 \\
Df & g/d & 0.95 & 0.89-1.01 & 0.13 & 0.96 & 0.85-1.07 & 0.43 & 0.94 & 0.86-1.04 & 0.22 & 0.91 & 0.76-1.09 & 0.30 \\
K & 10mg/d & 1.00 & 0.99-1.00 & 0.31 & 1.00 & 0.99-1.01 & 0.57 & 1.01 & 1.00-1.02 & 0.20 & 0.98 & 0.97-1.00 & 0.04 \\
Na & 100mg/d & 0.99 & 0.97-1.01 & 0.40 & 1.02 & 0.99-1.06 & 0.22 & 0.98 & 0.94-1.02 & 0.38 & 0.93 & 0.88-0.99 & 0.02 \\
Mg & 10mg/d & 0.98 & 0.94-1.02 & 0.37 & 0.99 & 0.93-1.06 & 0.82 & 1.03 & 0.96-1.10 & 0.38 & 0.91 & 0.81-1.02 & 0.09 \\
Ca & 100mg/d & 0.98 & 0.89-1.0

In [16]:
# 获取col,c1,951,952,df_origin的p
df_cox_final = df_cox_origin[['col','stage','c1','951','952']] 
# df_cox_final['p'] = df_origin['p']
df_cox_final['ci'] = df_cox_origin.apply(lambda row: f"{row['951']:.2f}-{row['952']:.2f}",axis=1)
df_cox_final['c1'] = df_cox_final['c1'].apply(lambda x : f'{x:.2f}')
df_cox_final['p'] = df_cox_origin['p'].astype(float)
df_cox_final['p'] = df_cox_final['p'].apply(lambda x: '***' if x < 0.001 else ('**' if x < 0.01 else ('*' if x < 0.05 else '')))
df_cox_final = df_cox_final[['col','stage','c1','ci','p']]
# col一列，替换成首字母大写的
df_cox_final['col'] = df_cox_final['col'].apply(lambda x : x.upper() if x == 'dpi' or x == 'dei' else x.capitalize())
df_cox_final.to_excel(f'{result_prefix}/res_cox_Albumin_final.xlsx')

/home/v-wuyueying/miniconda3/envs/esrd/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/v-wuyueying/miniconda3/envs/esrd/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/v-wuyueying/miniconda3/envs/esrd/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 